# PCA, K-means, and Hierarchical clustering of PLFA data of microbialites from Cenote Azul

**Author:** Julie Hartz </br>
**Date:** 24 October 2024

## Imports

In [31]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go 

## Data loading and pre-processing

In [32]:
chem711_df = pd.read_csv("../data/chem711.csv", index_col = 0)
chem711_df = chem711_df.iloc[0:18]
chem711_df = chem711_df[["CA06a", "CA06b", "CA06c", "CA10", "CA20", "CA30", "CA43", "CA49"]]
# Fill null values with 0
chem711_df = chem711_df.fillna(value=0)
# Transpose to have samples as rows
chem711_df = chem711_df.T
# Drop depth column
chem711_df = chem711_df.drop(columns=["depth"])
chem711_df

,total PLFA,mean point-wise dimension,stdev point-wise dimension,skewness point-wise dimension,kurtosis point-wise dimension,Methyltetradecanoate (7) 14:0,Methyl-13-methyltetradecanoate (8) i-15:0,Methyl-12-methyltetradecanoate (9) a-15:0,Methyl-14-methylpentadecanoate (13) i-16:0,Methyl-cis-9-hexadecenoate (14) 16:1,Methylhexadecanote (15) 16:0,Methyl-15-methylhexadecanoate (16) i-17:0,"Methyl-cis-9,10-methylenehexadecanoate (17) 17:2",Methylheptadecanoate (18) 17:0,"Methyl-cis-9,12-octadecadienoate (20) 18:2",Methyl-cis-9-octadecenoate (21) 18:1,Methyloctadecanoate (23) 18:0
CA06a,1.973970,4.610000e-07,1.440000e-07,0.117980,0.525763,4.749213,6.503959,3.214133,0.563217,4.073159,0.000,0.879606,0.000000,0.527707,2.739597,9.362593,8.112187
CA06b,1.824357,9.570000e-07,2.890000e-07,-0.510403,0.330758,5.290313,7.928223,5.626187,0.665504,2.984355,0.000,0.603548,0.265902,0.383450,2.975520,10.690997,6.161357
CA06c,2.320800,6.190000e-07,2.890000e-07,-0.510403,0.330758,6.441569,6.848231,5.618179,0.683622,5.240838,0.000,0.531455,0.396580,0.430296,4.119776,14.452348,7.184185
CA10,0.658871,7.940000e-07,2.360000e-07,0.588772,0.425150,0.000000,2.892000,2.140000,0.762000,0.904000,7.172,0.794000,0.798000,0.000000,3.919000,2.808000,4.166000
CA20,0.620281,5.000000e-07,1.840000e-07,0.421384,0.168211,0.000000,3.596000,2.443000,0.854000,0.867000,6.284,0.938000,0.828000,0.000000,3.855000,1.928000,3.219000
CA30,0.215621,1.520000e-06,1.490000e-07,0.185906,-0.537746,0.000000,1.313000,1.152000,0.457000,0.438000,1.665,0.452000,0.000000,0.000000,0.822000,0.891000,1.435000
CA43,0.273049,1.120000e-06,4.990000e-07,-0.336747,-0.501993,0.000000,0.545000,0.362000,0.114000,0.200000,3.456,0.140000,0.100000,1.750000,1.589000,1.716000,1.049000
CA49,0.135918,2.740000e-06,8.680000e-07,0.987111,0.504095,0.000000,0.056000,0.014000,0.000000,0.000000,0.917,0.000000,0.000000,1.167000,1.128000,1.210000,0.945000


In [33]:
# Only extract PLFA data (morphometrics will be for another analysis) and add group column (shallow vs deep)
plfa_dataset = chem711_df[chem711_df.columns[5:]]
new_columns = [column.split(" ")[2] for column in plfa_dataset.columns]
plfa_dataset["group"] = ["shallow", "shallow", "shallow", "shallow", "shallow", "deep", "deep", "deep"]
plfa_dataset

/var/folders/8r/4jrrmbyd2hsbp5tjhv6zd1c00000gn/T/ipykernel_88807/3505329165.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Methyltetradecanoate (7) 14:0,Methyl-13-methyltetradecanoate (8) i-15:0,Methyl-12-methyltetradecanoate (9) a-15:0,Methyl-14-methylpentadecanoate (13) i-16:0,Methyl-cis-9-hexadecenoate (14) 16:1,Methylhexadecanote (15) 16:0,Methyl-15-methylhexadecanoate (16) i-17:0,"Methyl-cis-9,10-methylenehexadecanoate (17) 17:2",Methylheptadecanoate (18) 17:0,"Methyl-cis-9,12-octadecadienoate (20) 18:2",Methyl-cis-9-octadecenoate (21) 18:1,Methyloctadecanoate (23) 18:0,group
CA06a,4.749213,6.503959,3.214133,0.563217,4.073159,0.000,0.879606,0.000000,0.527707,2.739597,9.362593,8.112187,shallow
CA06b,5.290313,7.928223,5.626187,0.665504,2.984355,0.000,0.603548,0.265902,0.383450,2.975520,10.690997,6.161357,shallow
CA06c,6.441569,6.848231,5.618179,0.683622,5.240838,0.000,0.531455,0.396580,0.430296,4.119776,14.452348,7.184185,shallow
CA10,0.000000,2.892000,2.140000,0.762000,0.904000,7.172,0.794000,0.798000,0.000000,3.919000,2.808000,4.166000,shallow
CA20,0.000000,3.596000,2.443000,0.854000,0.867000,6.284,0.938000,0.828000,0.000000,3.855000,1.928000,3.219000,shallow
CA30,0.000000,1.313000,1.152000,0.457000,0.438000,1.665,0.452000,0.000000,0.000000,0.822000,0.891000,1.435000,deep
CA43,0.000000,0.545000,0.362000,0.114000,0.200000,3.456,0.140000,0.100000,1.750000,1.589000,1.716000,1.049000,deep
CA49,0.000000,0.056000,0.014000,0.000000,0.000000,0.917,0.000000,0.000000,1.167000,1.128000,1.210000,0.945000,deep


## Data scaling

In [34]:
# Here chose a robust scaler, different normalizatoins and scalings can be tested in the interactive dashboard
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler()
data_scaled = transformer.fit_transform(plfa_dataset.drop(columns=["group"]))

## PCA analysis

In [35]:
from sklearn.decomposition import PCA
 
# Perform PCA on Scaled Data
pca = PCA(n_components=5)
 
 
pca_features = pca.fit_transform(data_scaled)
 
# Principal components correlation coefficients
loadings = pca.components_
 
# Number of features before PCA
n_features = pca.n_features_in_
 
# Feature names before PCA
feature_names = plfa_dataset.columns[0:-1]
 
# PC names
pc_list = [f'PC{i}' for i in list(range(1, n_features + 1))]
 
# Match PC names to loadings
pc_loadings = dict(zip(pc_list, loadings))
 
# Matrix of corr coefs between feature names and PCs
loadings_df = pd.DataFrame.from_dict(pc_loadings)
loadings_df['feature_names'] = feature_names
loadings_df = loadings_df.set_index('feature_names')

In [36]:
pca.explained_variance_ratio_

array([0.58050906, 0.30471811, 0.07173639, 0.02822795, 0.0100167 ])

In [37]:
loadings_df.sort_values(by="PC1", ascending=False)

,PC1,PC2,PC3,PC4,PC5
feature_names,,,,,
Methylheptadecanoate (18) 17:0,0.343764,-0.329127,-0.683294,0.326072,0.323703
Methylhexadecanote (15) 16:0,0.014893,0.481422,-0.352957,0.113476,-0.027744
"Methyl-cis-9,10-methylenehexadecanoate (17) 17:2",-0.193775,0.372085,-0.447239,-0.311610,-0.104024
Methyltetradecanoate (7) 14:0,-0.218436,-0.314029,-0.041541,-0.036242,0.008465
"Methyl-cis-9,12-octadecadienoate (20) 18:2",-0.246767,0.090614,-0.394907,-0.044147,-0.289924
Methyl-cis-9-octadecenoate (21) 18:1,-0.255596,-0.317484,-0.142447,-0.126380,-0.163978
Methyloctadecanoate (23) 18:0,-0.259757,-0.167899,-0.044776,0.342466,-0.205576
Methyl-13-methyltetradecanoate (8) i-15:0,-0.267017,-0.164756,-0.023327,0.062156,0.414705
Methyl-cis-9-hexadecenoate (14) 16:1,-0.285527,-0.304141,-0.070217,0.129688,-0.515244


## Visalization of PCA

In [38]:
import plotly.express as px
from plotly.subplots import make_subplots

pca = PCA(n_components=5)
Xt = pca.fit_transform(data_scaled)
y = target
text = [plfa.split(" ")[2] for plfa in loadings_df.index]

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=loadings_df["PC1"], y=loadings_df["PC2"], mode="markers+text", marker_size=1,
                         text=text, textposition="top left"), secondary_y=False)
for i in range(len(loadings_df["PC1"])):
    fig.add_annotation(ax = 0, axref = 'x', ay = 0, ayref = 'y', x = loadings_df["PC1"][i],
                       arrowcolor='blue', xref = 'x', y =loadings_df["PC2"][i],
                       yref='y', arrowwidth=1,arrowside='end',arrowsize=1,arrowhead = 4)
fig.update_annotations(opacity=0.3)

    
fig.update_layout(xaxis2= {'anchor': 'y', 'overlaying': 'x', 'side': 'top'},
                  yaxis_domain=[0, 0.94], width=700, height=600, showlegend=False,
                  margin=dict(t=0, b=0, l=0, r=0))

fig.add_trace(go.Scatter(x=Xt[:,0], y=Xt[:,1], mode="markers+text",
                         text=["CA06a", "CA06b", "CA06c", "CA10", "CA20", "CA30", "CA43", "CA49"],
                         textposition="top right",
                         marker_symbol= "circle",
                         marker_line_color="midnightblue", marker_color="lightskyblue",
                         marker_line_width=2, marker_size=10,), secondary_y=True)
fig.data[1].update(xaxis='x2')
fig.update_xaxes(title="PC1",)
fig.update_yaxes(title="PC2")
fig.show()

## K-means clustering

Similarly to the PCA, the k-means clustering clustered (i.e. labeld) the shallowest samples CA06a, b, and c, CA10, and CA20 together, and the deeper CA30, CA43, and CA49 samples together.

In [39]:
from sklearn.cluster import KMeans
import numpy as np

kmeans = KMeans(n_clusters=2, random_state=0, n_init="auto").fit(data_scaled)
kmeans.labels_
labels = ["shallow" if x == 0 else "deep" for x in kmeans.labels_]
labels

['shallow', 'shallow', 'shallow', 'shallow', 'shallow', 'deep', 'deep', 'deep']

In [40]:
px.scatter(data_frame=plfa_dataset, x="Methyl-13-methyltetradecanoate (8) i-15:0", y="Methyl-12-methyltetradecanoate (9) a-15:0", color=labels)

## Hierarchical clustering (dendrogram)

In [42]:
import plotly.graph_objects as go
import plotly.figure_factory as ff

import numpy as np
from scipy.spatial.distance import pdist, squareform


# get data
data_array = data_scaled
labels = ["CA06a", "CA06b", "CA06c", "CA10", "CA20", "CA30", "CA43", "CA49"]

# Initialize figure by creating upper dendrogram
fig = ff.create_dendrogram(data_array, orientation='bottom', labels=labels)
for i in range(len(fig['data'])):
    fig['data'][i]['yaxis'] = 'y2'

# Create Side Dendrogram
dendro_side = ff.create_dendrogram(data_array, orientation='right')
for i in range(len(dendro_side['data'])):
    dendro_side['data'][i]['xaxis'] = 'x2'

# Add Side Dendrogram Data to Figure
for data in dendro_side['data']:
    fig.add_trace(data)

# Create Heatmap
dendro_leaves = dendro_side['layout']['yaxis']['ticktext']
dendro_leaves = list(map(int, dendro_leaves))
data_dist = pdist(data_array)
heat_data = squareform(data_dist)
heat_data = heat_data[dendro_leaves,:]
heat_data = heat_data[:,dendro_leaves]

heatmap = [
    go.Heatmap(
        x = dendro_leaves,
        y = dendro_leaves,
        z = heat_data,
        colorscale = 'Blues'
    )
]

heatmap[0]['x'] = fig['layout']['xaxis']['tickvals']
heatmap[0]['y'] = dendro_side['layout']['yaxis']['tickvals']

# Add Heatmap Data to Figure
for data in heatmap:
    fig.add_trace(data)

# Edit Layout
fig.update_layout({'width':800, 'height':800,
                         'showlegend':False, 'hovermode': 'closest',
                         })
# Edit xaxis
fig.update_layout(xaxis={'domain': [.15, 1],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'ticks':""})
# Edit xaxis2
fig.update_layout(xaxis2={'domain': [0, .15],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Edit yaxis
fig.update_layout(yaxis={'domain': [0, .85],
                                  'mirror': False,
                                  'showgrid': False,
                                  'showline': False,
                                  'zeroline': False,
                                  'showticklabels': False,
                                  'ticks': ""
                        })
# Edit yaxis2
fig.update_layout(yaxis2={'domain':[.825, .975],
                                   'mirror': False,
                                   'showgrid': False,
                                   'showline': False,
                                   'zeroline': False,
                                   'showticklabels': False,
                                   'ticks':""})

# Plot!
fig.show()